In [16]:
import pandas as pd
import networkx as nx

# Caminhos dos arquivos
ihgb_path = 'IHGB.xlsx'
junta_academia_path = 'Junta-Academia.xlsx'

# Carregar os dados
ihgb_data = pd.ExcelFile(ihgb_path).parse("Sheet1")
junta_academia_data = pd.ExcelFile(junta_academia_path).parse("Sheet1")

# Ajustar índices para criar matrizes de adjacência
ihgb_matrix = ihgb_data.set_index("Lista de Códigos")
junta_matrix = junta_academia_data.set_index("Lista de Códigos")

# Identificar nós comuns
common_nodes = ihgb_matrix.index.intersection(junta_matrix.index)
ihgb_matrix = ihgb_matrix.loc[common_nodes, common_nodes]
junta_matrix = junta_matrix.loc[common_nodes, common_nodes]

# Criar grafos
ihgb_graph = nx.from_pandas_adjacency(ihgb_matrix, create_using=nx.Graph())
junta_graph = nx.from_pandas_adjacency(junta_matrix, create_using=nx.Graph())

# Combinar grafos
combined_graph = nx.Graph()

# Adicionar arestas do IHGB
for u, v, data in ihgb_graph.edges(data=True):
    weight1 = data.get("weight", 0)
    if junta_graph.has_edge(u, v):
        weight2 = junta_graph[u][v]["weight"]
        combined_graph.add_edge(u, v, weight=weight1 + weight2, origin="Both")
    else:
        combined_graph.add_edge(u, v, weight=weight1, origin="IHGB")

# Adicionar arestas exclusivas da Junta-Academia
for u, v, data in junta_graph.edges(data=True):
    if not combined_graph.has_edge(u, v):
        weight2 = data.get("weight", 0)
        combined_graph.add_edge(u, v, weight=weight2, origin="Junta-Academia")

# Adicionar atributos para os nós
for node in combined_graph.nodes():
    combined_graph.nodes[node]["label"] = node

# Salvar o grafo combinado como GML
output_path = "combined_graph_full_data.gml"
nx.write_gml(combined_graph, output_path)

output_path


'combined_graph_full_data.gml'